In [6]:
from tensorflow.keras import backend as K
K.clear_session()

In [7]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [8]:
!pip install tensorflow==2.12.0

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 723.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 MB 14.2 MB/s eta 0:00:00
  

In [9]:
!pip install numpy==1.23.5
!pip install keras==2.12.0
!pip install tensorboard==2.12.0
!pip install tensorflow-text==2.12.0
!pip install tensorflow-tpu==2.12.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 42.9 MB/s eta 0:00:00
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.0.0
    Uninstalling google-auth-oauthlib-1.0.0:
      Successfully uninstalled google-auth-oauthlib-1.0.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.3
    Uninstalling tensorboard-2.12.3:
      Successfully uninstalled tensorboard-2.12.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-tpu 2.18.0 requires keras>=3.5.0, but you have keras 2.12.0 which is incompatible.
tensorflow-tpu 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
tensorflow-tpu 2.18.0 requires tensorboard<2.19,>=2.18, but you have tensorboard 2.12.0 

In [2]:
# Cell 1: Import Libraries
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [3]:
DATASET_DIR = '/content/drive/MyDrive/Untitled folder/dataset'


In [4]:
# Cell 2: Constants and Dataset Loading
# Constants
IMAGE_SIZE = 224
EPOCHS = 20
BATCH_SIZE = 32

# Load Dataset
def load_dataset():
    images = []
    labels = []
    gesture_names = os.listdir(DATASET_DIR)
    gesture_names.sort()
    label_dict = {name: idx for idx, name in enumerate(gesture_names)}

    for gesture_name in gesture_names:
        gesture_folder = os.path.join(DATASET_DIR, gesture_name)
        for img_name in os.listdir(gesture_folder):
            img_path = os.path.join(gesture_folder, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
            img = img / 255.0  # Normalize
            images.append(img)
            labels.append(label_dict[gesture_name])

    return np.array(images), to_categorical(labels), label_dict

print("[INFO] Loading dataset...")
X, y, label_dict = load_dataset()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


[INFO] Loading dataset...


In [5]:
# Cell 3: Build Model
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(label_dict), activation='softmax')
    ])
    return model

print("[INFO] Building model...")
model = build_model()
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

[INFO] Building model...


In [6]:
# Cell 4: Train Model
print("[INFO] Training model...")
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=EPOCHS, batch_size=BATCH_SIZE)

[INFO] Training model...
Epoch 1/20
141/141 [==============================] - 382s 3s/step - loss: 0.9448 - accuracy: 0.7304 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 2/20
141/141 [==============================] - 377s 3s/step - loss: 0.1123 - accuracy: 0.9605 - val_loss: 3.7160e-04 - val_accuracy: 1.0000
Epoch 3/20
141/141 [==============================] - 372s 3s/step - loss: 0.0763 - accuracy: 0.9735 - val_loss: 7.7863e-05 - val_accuracy: 1.0000
Epoch 4/20
141/141 [==============================] - 374s 3s/step - loss: 0.0543 - accuracy: 0.9799 - val_loss: 4.6778e-04 - val_accuracy: 1.0000
Epoch 5/20
141/141 [==============================] - 373s 3s/step - loss: 0.0562 - accuracy: 0.9802 - val_loss: 0.0014 - val_accuracy: 0.9991
Epoch 6/20
141/141 [==============================] - 373s 3s/step - loss: 0.0374 - accuracy: 0.9886 - val_loss: 0.0016 - val_accuracy: 0.9991
Epoch 7/20
141/141 [==============================] - 373s 3s/step - loss: 0.0324 - accuracy: 0.9893 - va

In [7]:
# Cell 5: Save Model
model.save('sign_language_model.h5')
print("[INFO] Model saved as 'sign_language_model.h5'")
print(f"[INFO] Label mapping: {label_dict}")


[INFO] Model saved as 'sign_language_model.h5'
[INFO] Label mapping: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'Hello': 8, 'I': 9, 'I Love You': 10, 'J': 11, 'K': 12, 'L': 13, 'M': 14, 'N': 15, 'O': 16, 'P': 17, 'Q': 18, 'R': 19, 'S': 20, 'T': 21, 'Thank You': 22, 'U': 23, 'V': 24, 'W': 25, 'X': 26, 'Y': 27}


In [8]:
# Evaluate model on test data
loss, accuracy = model.evaluate(X_test, y_test)

print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Test Loss: {loss:.4f}")


36/36 [==============================] - 21s 583ms/step - loss: 1.3302e-04 - accuracy: 1.0000
Test Accuracy: 100.00%
Test Loss: 0.0001


In [9]:
model.save("/content/drive/My Drive/Untitled folder/sign_language_model.h5")  # Saves the model as a single file
